<a href="https://colab.research.google.com/github/Intelligence-Engineering-LAB-KU/Seminar/blob/master/summer_2020/0728_wschoi_CNN_with_or_without_BatchNorm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 라이브러리 로드

필요한 라이브러리를 로드해줍니다.

In [1]:
from __future__ import print_function
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

## 2. 데이터셋 로드

[CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) 데이터를 사용할 예정입니다. 

CIFAR-10에 대한 데이터 살펴보기 예제는 [이 포스트](https://github.com/Intelligence-Engineering-LAB-KU/Deeplearning-Tutorial/blob/master/Tutorial_2_Sobel_Opearatior.ipynb)를 살펴봐주세요.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

## 3. 클래스 이진코드화

[해당 포스트](https://github.com/Intelligence-Engineering-LAB-KU/Deeplearning-Tutorial/blob/master/Tutorial_2_Sobel_Opearatior.ipynb)에서와 같이, 클래스를 이진코드화 해보겠습니다.

In [3]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)

print('y_train shape (이진화 전)', y_train.shape)
print('y_train[0]:', y_train[0])

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('y_train shape (이진화 후)', y_train.shape)
print('y_train[0]:', y_train[0])

y_train shape (이진화 전) (50000, 1)
y_train[0]: [6]
y_train shape (이진화 후) (50000, 10)
y_train[0]: [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


### 4. CIFAR10 분류 CNN 모델 생성

4장에서는 Keras의 [Conv2d API](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) 등을 이용하여 CIFAR-10 분류를 위한 CNN 기반 모델을 정의해보겠습니다. 

```python
tf.keras.layers.Conv2D(
    filters, kernel_size, strides=(1, 1), padding='valid', data_format=None,
    dilation_rate=(1, 1), activation=None, use_bias=True,
    kernel_initializer='glorot_uniform', bias_initializer='zeros',
    kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None,
    kernel_constraint=None, bias_constraint=None, **kwargs
)
```
#### 4.1. Batch Norm 이 없는 모델

다음과 같은 모델을 구현해보겠습니다.

![](https://imgur.com/SDxHEhR.png)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from keras.regularizers import l1

tf.random.set_seed(2020)

model = Sequential([
    Conv2D(filters=2,kernel_size=(3,3), padding='same', activation='relu', 
           input_shape=(32, 32, 3)),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=4,kernel_size=(3,3), padding='same', activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=8,kernel_size=(3,3), padding='same', activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=16,kernel_size=(3,3), padding='same', activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=32,kernel_size=(3,3), padding='same', activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=64,kernel_size=(3,3), padding='same', activation='relu'),
    Flatten(),
    Dense(10, activation='softmax')])

Using TensorFlow backend.



이제 BatchNorm이 없는 모델을 학습시켜보겠습니다.

In [5]:
tf.random.set_seed(2020)

opt = keras.optimizers.Adam()

# 모델 생성
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

norm_x_train = x_train.astype('float32')
norm_x_test = x_test.astype('float32')
norm_x_train /= 255
norm_x_test /= 255

model.fit(norm_x_train, y_train,
          batch_size=32,
          epochs=10,
          validation_data=(norm_x_test, y_test),
          shuffle=True)

scores = model.evaluate(norm_x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/10
1563/1563 [==============================] - 35s 23ms/step - loss: 1.9076 - accuracy: 0.2666 - val_loss: 1.7090 - val_accuracy: 0.3555
Epoch 2/10
1563/1563 [==============================] - 37s 24ms/step - loss: 1.6619 - accuracy: 0.3758 - val_loss: 1.6284 - val_accuracy: 0.3920
Epoch 3/10
1563/1563 [==============================] - 37s 24ms/step - loss: 1.5942 - accuracy: 0.4089 - val_loss: 1.5536 - val_accuracy: 0.4243
Epoch 4/10
1563/1563 [==============================] - 37s 23ms/step - loss: 1.5433 - accuracy: 0.4329 - val_loss: 1.5119 - val_accuracy: 0.4448
Epoch 5/10
1563/1563 [==============================] - 36s 23ms/step - loss: 1.5067 - accuracy: 0.4478 - val_loss: 1.4775 - val_accuracy: 0.4590
Epoch 6/10
1563/1563 [==============================] - 36s 23ms/step - loss: 1.4768 - accuracy: 0.4628 - val_loss: 1.4609 - val_accuracy: 0.4634
Epoch 7/10
1563/1563 [==============================] - 36s 23ms/step - loss: 1.4505 - accuracy: 0.4747 - val_loss: 1.4300 -

#### 4.2. Batch Norm 이 있는 모델

이번에는 4.1 모델과 같되, 중간중간 BatchNorm이 있는 모델을 구현하여 학습시켜보겠습니다.

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, BatchNormalization
from keras.regularizers import l1

tf.random.set_seed(2020)

model_with_bn = Sequential([
    BatchNormalization(input_shape=(32, 32, 3)),
    Conv2D(filters=2,kernel_size=(3,3), padding='same', activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    BatchNormalization(),
    Conv2D(filters=4,kernel_size=(3,3), padding='same', activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    BatchNormalization(),
    Conv2D(filters=8,kernel_size=(3,3), padding='same', activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    BatchNormalization(),
    Conv2D(filters=16,kernel_size=(3,3), padding='same', activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    BatchNormalization(),
    Conv2D(filters=32,kernel_size=(3,3), padding='same', activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    BatchNormalization(),
    Conv2D(filters=64,kernel_size=(3,3), padding='same', activation='relu'),
    Flatten(),
    BatchNormalization(),
    Dense(10, activation='softmax')])

opt = keras.optimizers.Adam()

model_with_bn.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model_with_bn.fit(norm_x_train, y_train,
          batch_size=32,
          epochs=10,
          validation_data=(norm_x_test, y_test),
          shuffle=True)

scores = model_with_bn.evaluate(norm_x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/10
1563/1563 [==============================] - 56s 36ms/step - loss: 1.8122 - accuracy: 0.3364 - val_loss: 1.6081 - val_accuracy: 0.4074
Epoch 2/10
1563/1563 [==============================] - 55s 35ms/step - loss: 1.5783 - accuracy: 0.4244 - val_loss: 1.4937 - val_accuracy: 0.4491
Epoch 3/10
1563/1563 [==============================] - 54s 34ms/step - loss: 1.4912 - accuracy: 0.4599 - val_loss: 1.4235 - val_accuracy: 0.4822
Epoch 4/10
1563/1563 [==============================] - 53s 34ms/step - loss: 1.4302 - accuracy: 0.4834 - val_loss: 1.3856 - val_accuracy: 0.5033
Epoch 5/10
1563/1563 [==============================] - 53s 34ms/step - loss: 1.3834 - accuracy: 0.5010 - val_loss: 1.3707 - val_accuracy: 0.5096
Epoch 6/10
1563/1563 [==============================] - 54s 35ms/step - loss: 1.3502 - accuracy: 0.5150 - val_loss: 1.3391 - val_accuracy: 0.5197
Epoch 7/10
1563/1563 [==============================] - 54s 34ms/step - loss: 1.3239 - accuracy: 0.5244 - val_loss: 1.3405 -



## Reference

- [cs231n](http://cs231n.github.io/)
- [Keras tutorial on CIFAR-10](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py)